In [1]:
import os
import csv
import pandas as pd
from string import punctuation
from collections import Counter
from nltk import *
from nltk.corpus import wordnet as wn

In [2]:
count = 1
for filenames in os.walk('../data/wikipedia_txt'):
    for filename in filenames[2]:
        text = open('../data/wikipedia_txt/'+filename, 'rU').read()
        text = text.replace('--', ' ')
        wt = word_tokenize(text.lower())
        p_tag = pos_tag(wt)
        word_tag = [(w[0].strip(punctuation), w[1]) for w in p_tag if len(w[0].strip(punctuation))>0]
        
        word_ext = []
        for word in word_tag:
            if word[1] == 'VB' and len(wn.synsets(word[0], pos=wn.VERB)) > 0:
                word_ext.append(word[0])
            elif word[1] in ['NN', 'NNS', 'NNP']:
                if word[1] == 'NNS' and wn.morphy(word[0], wn.NOUN) != None:
                    word_nn = wn.morphy(word[0], wn.NOUN)
                elif word[1] in ['NN', 'NNP'] and len(wn.synsets(word[0], pos=wn.NOUN)) > 0:
                    word_nn = word[0]
                if len(wn.synsets(word_nn, pos=wn.NOUN)) == 0:
                    word_ext.append(word[0])
                else:    
                    [word_ext.append((n.hypernyms()[0]).name().split('.')[0]) for n in wn.synsets(word_nn, pos=wn.NOUN) if len(n.hypernyms())!=0]
            elif word[1] in ['VBG', 'VBZ', 'VBD', 'VBN']:
                word_ext.append(wn.morphy(word[0], wn.VERB))
            elif word[1] == 'JJ' and wn.morphy(word[0], wn.ADJ) != None and len(wn.synsets(word[0], pos=wn.ADJ)) > 0:
                word_ext.append(wn.morphy(word[0], wn.ADJ))
            elif word[1] == 'RB' and wn.morphy(word[0], wn.ADV) != None and len(wn.synsets(word[0], pos=wn.ADV)) > 0:
                word_ext.append(wn.morphy(word[0], wn.ADV))

        word_count = Counter(word_ext).most_common()

        with open('unigram_count.csv', 'a+') as csv_file:
            writer = csv.writer(csv_file) 
            for w in word_count:
                writer.writerow([filename[:-4], w[0], w[1]])
        
        if count % 30 == 0 or count == 2974:
            if count == 2974:
                count=3000
            print('*'*int(count/30)+'-'*(100-int(count/30))+' '+str(int(count/30))+'%', end='\r', flush=True)
        count += 1

/root/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:4: DeprecationWarning: 'U' mode is deprecated


In [5]:
wc_file = pd.read_csv('unigram_count.csv', names = ['article', 'word', 'count'])
wc_file[:5]

,article,word,count
0,Fox_Television_Studios,broadcasting,10
1,Fox_Television_Studios,receiver,10
2,Fox_Television_Studios,workplace,10
3,Fox_Television_Studios,algonquian,10
4,Fox_Television_Studios,telecommunication_system,9


In [6]:
labels_file = pd.read_csv('../data/comp5046-labels.csv', index_col='article', names = ['web','article','label'], usecols=['article','label'] )
labels_file[:5]

,label
article,
WWEG,Entertainment
Vitamin_String_Quartet,Entertainment
Vidyasagar_Evening_College,Education
University_of_Belgrade_Faculty_of_Philosophy,Education
Today's_Best_Hits,Entertainment


In [7]:
word_table = wc_file.pivot_table(values='count', index='article', columns='word', fill_value=0)
word_table[:5]

word,0,1,10,1000th,100th,101st,10th,115,115th,11th,...,zambian,zealous,zimbabwean,zionist,zip,zonal,zone,zoological,zoology,zoonosis
article,,,,,,,,,,,,,,,,,,,,,
102.1_Bay_Radio,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
107_Squadron_(Israel),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
109th_Infantry_Division_(German_Empire),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10th_Division_(German_Empire),0,0,0,0,0,0,6,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10th_Parachute_Battalion_(United_Kingdom),0,0,0,0,0,0,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
data = pd.merge(labels_file, word_table, left_index = True, right_index = True)
data[:5]

,label_x,0,1,10,1000th,100th,101st,10th,115,115th,...,zambian,zealous,zimbabwean,zionist,zip,zonal,zone,zoological,zoology,zoonosis
article,,,,,,,,,,,,,,,,,,,,,
102.1_Bay_Radio,Entertainment,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
107_Squadron_(Israel),Public administration,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
109th_Infantry_Division_(German_Empire),Public administration,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10th_Division_(German_Empire),Public administration,0,0,0,0,0,0,6,0,0,...,0,0,0,0,0,0,0,0,0,0
10th_Parachute_Battalion_(United_Kingdom),Public administration,0,0,0,0,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
data.to_csv('unigram_training_data.csv')